<a href="https://colab.research.google.com/github/PanoEvJ/Building_with_LLMs/blob/main/Building_a_Simple_Classifier_with_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align = "center" draggable=”false” ><img src="https://user-images.githubusercontent.com/37101144/161836199-fdb0219d-0361-4988-bf26-48b0fad160a3.png" 
     width="200px"
     height="auto"/>
</p>

# Building a Simple Classifier with LLMs


---

Before we delve too deep into the powerful tool that LLMs are, let's take a moment to remember that most of the models in production are still leveraging traditional Supervised Machine Learning Workflows.

In this notebook we'll cover the following topics:

- [ ] Data Preparation
- [ ] Fine-tuning A Model for Text Classification With 🤗

### Check if notebook is run in Colab


In [35]:
from IPython import get_ipython

if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  colab = True
else:
  print('Not running on CoLab')
  colab = False

Not running on CoLab


### Install requirements

In [36]:
if not colab:
    !python -m pip install -r requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.0
    Uninstalling transformers-4.28.0:
      Successfully uninstalled transformers-4.28.0


## Task 1: Prepare the Data

The [20 Newsgroups](http://qwone.com/~jason/20Newsgroups/) data set is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups.

Here are some [details](https://scikit-learn.org/stable/datasets/real_world.html#newsgroups-dataset) about the dataset from Scikit Learn!

Let's load the data and get it into a usable format!

In [1]:
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Have a look around the data! Take note of things like data types, column names, and everything else!

Also take note of how many classes there are in our labels and mark it down in the cell below!

In [2]:
NUM_LABELS = 20

Now that we have our raw data - let's convert that into some `pd.Series` objects using `pandas`!

In [3]:
import pandas as pd

X, y = pd.Series(train.data), pd.Series(train.target)
X_test, y_test = pd.Series(test.data), pd.Series(test.target)

Now let's get the Hugging Face `datasets` (documentation [here](https://huggingface.co/docs/datasets/index)) library so we can convert our data into a more usable format.

In [37]:
if colab: 
    !pip install -U -q datasets

We'll want to first convert our separate series objects into a combined `pd.DataFrame` with columns: `text` and `label`, for our Xs and ys respectively. 

After that, it's as easy as loading the `pd.DataFrame` into a `Dataset` object!

Don't forget to refer to the documentation if you're stuck!

In [5]:
from datasets import Dataset

train_df = pd.DataFrame({
    'text': X,
    'label': y
})

test_df = pd.DataFrame({
    'text': X_test,
    'label': y_test
})

train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

/home/pevj/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now we can cast our label columns to `datasets.features.ClassLabel` objects using `class_encode_column`! (documentation [here](https://huggingface.co/docs/datasets/package_reference/main_classes#datasets.Dataset.class_encode_column))

In [6]:
train_ds = train_ds.class_encode_column('label') 
test_ds =  test_ds.class_encode_column('label') 

Let's split our training data into training/validation sets using `datasets` `ds.train_test_split()` method! We're going to hold **10%** of the data for validation.

Be sure to stratify by the labels, and set your seed to 19. 

This will return a `DatasetDict`, which we'll assign to `data_dsd`.


Again, if needed: [Documentation!](https://huggingface.co/docs/datasets/package_reference/main_classes#datasets.Dataset.train_test_split)

In [11]:
data_dsd = train_ds.train_test_split(test_size=0.10, stratify_by_column='label', seed=19)

Now we need to do some reworking so our `DatasetDict` has all the required splits.

In [12]:
data_dsd['validation'] = data_dsd['test']
data_dsd['test'] = test_ds

In [14]:
data_dsd

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10182
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7532
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1132
    })
})

## TASK 2: Fine-tuning a pre-trained model via 🤗

In this task, we'll be fine-tuning a simple classifier for our above data using Hugging Face's `transformers` library (documentation [here](https://huggingface.co/docs/transformers/index)) as well as the `accelerate` library. (documentation [here](https://huggingface.co/docs/accelerate/index))

Before we dive in, let's take a pit stop to discuss what fine-tuning is - in broad strokes.

- Fine-tuning is a transfer learning approach where a pre-trained machine learning model is further trained on new data, often to specialize in a certain task. This process can involve training the entire network or only a subset of it, with untrained layers remaining 'frozen'.

- This method is prevalent in Natural Language Processing (NLP) and convolutional neural networks. In the latter, early layers capturing lower-level features are typically frozen, while in NLP, large models like GPT-2 are fine-tuned for specific tasks, improving their performance. However, full fine-tuning can be computationally costly and might lead to overfitting.

- Although fine-tuning is commonly executed through supervised learning, it can also be done using weak supervision or reinforcement learning. For instance, language models like ChatGPT and Sparrow are fine-tuned using reinforcement learning from human feedback.

Okay, so now that we have had a brief overview of what fine-tuning actually is - let's set ourselves up to do some!

In [15]:
if colab: 
    !pip install -U -q transformers==4.28.0 accelerate

Now that we have our dependencies - we're going to need to choose a base model! 

While there are a number of great choices (and feel free to improvise here), we're going to be sticking with the classic: 

DistilBERT (uncased)!

- [Paper](https://arxiv.org/pdf/1910.01108.pdf)
- [Model Card](https://huggingface.co/distilbert-base-uncased)

Let's begin by setting up our deisred model check-point, which we'll pull from the Hugging Face Hub!

In [16]:
model_checkpoint = "distilbert-base-uncased"

Next up - we'll want to ensure that our text is converted into a compatible format for use in fine-tuning our model:

Enter, Tokenization!

We'll be leveraging Hugging Face's `AutoTokenizer` to make this part a breeze! (documentation [here](https://huggingface.co/docs/transformers/v4.20.1/en/model_doc/auto#transformers.AutoTokenizer))

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Those who read the model card/paper will have noticed that DistilBERT has a maximum input length of 512, so we'll need to make sure our inputs never exceed that length. Let's set up a parameter for our maximum sequence length, as well as create a preprocessing function to truncate any potentially long sequences.

In [19]:
MAX_LEN = 256

def preprocess_function(examples):
    return tokenizer(examples["text"], max_length=MAX_LEN, truncation=True, padding=True)

Now we can use our work from earlier to `map` our preprocessing function over our dataset! (documentation [here](https://huggingface.co/docs/datasets/package_reference/main_classes#datasets.Dataset.map))

In [20]:
tokenized_text = data_dsd.map(preprocess_function, batched=True)

In [21]:
tokenized_text

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10182
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 7532
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1132
    })
})

We'll be training in TensorFlow - so we're going to want to create our data collator with `return_tensors='tf'`, otherwise we just want to set up our `DataCollatorWithPadding` with our tokenizer.

In [27]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer, return_tensors='tf')

Now we can set up our `tf_datasets`!

We'll want to include the `attention_mask`, `input_ids`, and `label` for each set - as well as shuffling the training set.

In [31]:
import tensorflow as tf

BATCH_SIZE = 16

tf_train_set = tokenized_text["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
)
tf_validation_set = tokenized_text["validation"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
    )
tf_test_set = tokenized_text["test"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
    )

ImportError: Called a Tensorflow-specific function but Tensorflow is not installed.

In [93]:
tf_train_set

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

Now we can set some hyper parameters, and our optimizer!

In [78]:
from transformers import create_optimizer

EPOCHS = 3
batches_per_epoch = len(tokenized_text["train"]) // BATCH_SIZE 
total_train_steps = int(batches_per_epoch * EPOCHS)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

Next up we can load our DistilBERT model using TFAutoModelForSequenceClassification - remember to set the `num_labels` correctly!

In [79]:
from transformers import TFAutoModelForSequenceClassification

my_bert = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_projector', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_39', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

We're going to use the naive example of `accuracy` for this notebook - but feel free to use whatever metric you believe will work best.

In [84]:
my_bert.compile(optimizer=optimizer, metrics=['accuracy'])

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


It's finally time for `model.fit`, let's go!

In [85]:
%%time
my_bert.fit(tf_train_set, validation_data=tf_validation_set)

637/637 [==============================] - 89s 108ms/step - loss: nan - accuracy: 0.0424 - val_loss: nan - val_accuracy: 0.0424
CPU times: user 1min 31s, sys: 3.78 s, total: 1min 35s
Wall time: 1min 28s


Once we've trained it - we can measure its performance on the test set. 

In [111]:
bert_loss, bert_acc = my_bert.compute_loss(tf_test_set), my_bert.evaluate(tf_test_set)

471/471 [==============================] - 10s 22ms/step - loss: nan - accuracy: 0.0424


Now that we have a model we're satisfied with - let's push it to the Hugging Face Hub!

In [112]:
from huggingface_hub import notebook_login

notebook_login()

In [115]:
MODEL_NAME = 'Bert-Classifer-News-Articles'
HUGGINGFACE_ACCT_NAME = 'PanoEvJ'

my_bert.push_to_hub(f"{HUGGINGFACE_ACCT_NAME}/{MODEL_NAME}")
tokenizer.push_to_hub(f"{HUGGINGFACE_ACCT_NAME}/{MODEL_NAME}")

tf_model.h5:   0%|          | 0.00/268M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/PanoEvJ/Bert-Classifer-News-Articles/commit/9b1d9e53753cfc92e0c0f4a1af29ecde901fea40', commit_message='Upload tokenizer', commit_description='', oid='9b1d9e53753cfc92e0c0f4a1af29ecde901fea40', pr_url=None, pr_revision=None, pr_num=None)

You're done! Now your model should be on the hub and you can edit/improve your model card! 🚀